In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [3]:
PATH = "/data/"

In [4]:
!ls {PATH}

models	sample_submission.csv  test.csv  tmp  train.csv


In [180]:
ord_df = pd.read_csv('{}train.csv'.format(PATH))
test_df = pd.read_csv('{}test.csv'.format(PATH))

In [184]:
ord_df.comment_text = ord_df.comment_text.str.replace('\n', '\t')
test_df.comment_text = test_df.comment_text.str.replace('\n', '\t')

In [185]:

trn_size = int(len(ord_df)*0.95 )
train_df = ord_df.iloc[:trn_size]
val_df = ord_df.iloc[trn_size:]
# sub = pd.read_csv('{}sample_submission.csv'.format(PATH))

In [186]:
train_df[['comment_text', 'toxic']].to_csv('{}trn.csv'.format(PATH), index=False, header=False)
val_df[['comment_text', 'toxic']].to_csv('{}val.csv'.format(PATH), index=False, header=False)

In [156]:
train_df[['comment_text', 'toxic']].iloc[0]['comment_text']

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [9]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)


In [12]:
bs=64; bptt=100

In [13]:
md = LanguageModelData.from_dataframes(PATH, field=TEXT, col='comment_text',train_df=train_df, val_df=val_df, test_df=test_df, bs=bs, bptt=bptt, min_freq=10)

In [14]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [15]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)


(2001, 26420, 1, 12816548)

In [16]:
TEXT.vocab.itos[:12]


['<unk>', '<pad>', '.', 'the', ',', '"', 'to', 'i', 'of', 'and', 'you', 'a']

In [17]:
TEXT.vocab.stoi['the']


3

In [18]:
md.trn_ds[0].text[:12]


['explanation',
 '\n',
 'why',
 'the',
 'edits',
 'made',
 'under',
 'my',
 'username',
 'hardcore',
 'metallica',
 'fan']

In [19]:
TEXT.numericalize([md.trn_ds[0].text[:12]])


Variable containing:
   698
    25
    93
     3
   148
   152
   198
    45
   673
  4589
 11214
  1265
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

In [20]:
next(iter(md.trn_dl))


(Variable containing:
    698     17    455  ...       0    246   7607
     25     28      2  ...       4      4      2
     93      0      7  ...       0      9    212
         ...            ⋱           ...         
    148      4    455  ...      63      4     53
    385     49      2  ...      31      9   1189
      8      3      7  ...     953    590   5461
 [torch.cuda.LongTensor of size 111x64 (GPU 0)], Variable containing:
     25
     28
      2
   ⋮   
    572
     65
   2222
 [torch.cuda.LongTensor of size 7104 (GPU 0)])

In [21]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [22]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))


In [23]:

learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [24]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                                
    0      4.911529   4.862442  
    1      4.480591   4.455935                                
    2      4.359549   4.362613                                
    3      4.313448   4.30938                                 
    4      4.212308   4.219637                                
    5      4.140606   4.158195                                
    6      4.11218    4.146964                                
    7      4.198393   4.189805                                
    8      4.153597   4.157058                                
    9      4.128773   4.122213                                
    10     4.06089    4.089012                                
    11     4.056622   4.062497                                
    12     3.997402   4.039912                                
    13     3.97386    4.03283                                 
    14     3.966023   4.026577                                



[4.0265765]

In [25]:
learner.save_encoder('adam1_enc')

In [26]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [46]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss                                
    0      4.087397   4.103743  
    1      4.064245   4.083046                                
    2      4.044287   4.064961                                
    3      4.017461   4.044334                                
    4      3.996781   4.028334                                
    5      3.955606   4.005434                                
    6      3.934688   3.990637                                
    7      3.910462   3.983895                                
    8      3.897824   3.9792                                  
    9      3.891785   3.978564                                



[3.9785643]

In [47]:
learner.save_encoder('adam3_enc')

In [48]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

# Test

In [73]:
m=learner.model
ss="""i'm not sure what you mean"""
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

"i 'm not sure what you mean"

In [82]:
def sample_model(m, s, l=50):
    def num_str(s): return TEXT.numericalize([proc_str(s)])
    def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs
# sample_model(m,'')

In [ ]:
train_df.iterrows()


In [76]:
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

i'm not sure what you mean 

by the way , the article is not about the subject .   it is not a personal attack .   it is not a personal attack .   it is not a personal attack .   it is a personal attack .   it is not a personal attack ...


# Sentiment

In [128]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))
LABELS = data.Field(sequential=False)

In [196]:
splits = data.TabularDataset.splits(
     path=PATH, train='trn.csv',
     validation='trn.csv', test='test.csv', format='csv',
     fields=[('text', TEXT), ('label', LABELS)])

In [198]:
t = splits[0].examples[10]
t.label, ' '.join(t.text[:16])

('0',
 '" \t fair use rationale for image : wonju.jpg \t\t thanks for uploading image : wonju.jpg')

In [199]:
md2 = TextData.from_splits(PATH, splits, bs)


In [200]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'adam3_enc')

In [201]:
m3.clip=25.
lrs=np.array([1e-4,1e-3,1e-2])

In [202]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  0%|          | 0/2368 [00:00<?, ?it/s]


TypeError: '<' not supported between instances of 'Example' and 'Example'